In [1]:
%matplotlib qt
import src
from src.sweep0d import Sweep0D
from src.sweep1d import Sweep1D
from src.sweep2d import Sweep2D
from src.sweep_queue import SweepQueue
from src.daq_driver import Daq, DaqAOChannel, DaqAIChannel
from src.util import init_database
from src.tracking import *
import nidaqmx
import time
import numpy as np
import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes import initialise_or_create_database_at
from qcodes.dataset.data_set import load_by_run_spec
from qcodes.dataset.data_export import get_data_by_id
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860

C:\Users\erunb\Anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\database.py:13: UserWarning: The module `qcodes.dataset.database` is deprecated.
Public features are available at the import of `qcodes`.
Private features are available in `qcodes.dataset.sqlite.*` modules.
  warnings.warn('The module `qcodes.dataset.database` is deprecated.\n'


In [2]:
daq = Daq("Dev2", "Daq")

In [3]:
forward_gate = Sweep1D(daq.ao0.voltage, -2, 2, .005, inter_delay=.1, x_axis_time=0)
backward_gate = Sweep1D(daq.ao0.voltage, 2, -2, -.02, inter_delay=.1, x_axis_time=0)
forward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ao1.voltage)
backward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ao1.voltage)

In [4]:
db = "Testing queue"
exps = []
samples = []

In [5]:
sweep_queue = SweepQueue(inter_delay = 1)

sweep_queue.append(forward_gate)
exps.append('B=0')
samples.append('forward gate')
sweep_queue.append(backward_gate)
exps.append('B=0')
samples.append('backward gate')

for x in np.linspace(1,5,6):
    # Step the B field
    B_step = Sweep1D(daq.ao1.voltage, x-1, x, .01, inter_delay=.1, x_axis_time=0)
    B_step.follow_param(daq.ai0.voltage, daq.ai1.voltage)
    # Add it to our queue
    sweep_queue.append(B_step)
    exps.append('B ramp')
    samples.append(f'ramp B to {x}')
    
    # Add new gate sweeps at that voltage
    forward_gate = Sweep1D(daq.ao0.voltage, -2, 2, .005, inter_delay=.1, x_axis_time=0)
    backward_gate = Sweep1D(daq.ao0.voltage, 2, -2, -.02, inter_delay=.1, x_axis_time=0)
    forward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ao1.voltage)
    backward_gate.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ao1.voltage)
    sweep_queue.append(forward_gate, backward_gate)
    exps.append(f'B={x}')
    samples.append('forward gate')
    exps.append(f'B={x}')
    samples.append('backward gate')
    
sweep_queue.load_database_info(db, exps, samples)

In [6]:
sweep_queue.start()

Starting sweeps
Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|██████████| 1/1 [00:00<00:00, 251.01it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|██████████| 1/1 [00:00<00:00, 250.30it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v7 -> v8: 100%|██████████| 1/1 [00:00<00:00, 333.52it/s]
Starting sweep of ao0 Voltage from -2               V to 2 V
Ramping to our starting setpoint value of -2 V
Done ramping ao0 Voltage to -2
Sweeping ao0 Voltage to 2 V
Starting experimental run with id: 1. 
Done with the sweep, ao0 Voltage=1.999999999999959
Finished sweep of ao0 Voltage from 2               V to -2 V
Starting sweep of ao0 Voltage from 2                   V to -2 V
Ramping to our starting setpoint value of 2 V
Sweeping ao0 Voltage to 2 V
Ramping ao0 Voltage to 2 . . . 
Done with the sweep, ao0 Voltage=1.999999999999959
Done ramp

Ramping to our starting setpoint value of 2 V
Sweeping ao0 Voltage to 2 V
Ramping ao0 Voltage to 2 . . . 
Done with the sweep, ao0 Voltage=1.999999999999959
Done ramping ao0 Voltage to 2
Sweeping ao0 Voltage to -2 V
Starting experimental run with id: 20. 
Done with the sweep, ao0 Voltage=-2.0000000000000027
Finished sweep of ao0 Voltage from -2               V to 2 V
Finished all sweeps!


In [ ]:
sweep_queue.is_running